In [1]:
!pip install vecstack 

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19877 sha256=96e74b01bad7199145a6dfbc00cb6201753d2ae5c45486e7c6772d13f9c0c167
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
pip install imblearn 

In [3]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")  

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


trainfile = r'/gdrive/My Drive/Colab Notebooks/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Colab Notebooks/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape) 
print(train_data.head()) 

print(test_data.shape) 
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...    

In [5]:
df=train_data.select_dtypes(exclude=[np.number])
print(df)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[65000 rows x 0 columns]


In [0]:
trainData_Copy = train_data.iloc[:, :-1].copy() 
testData_Copy = test_data.iloc[:, :-1].copy() 

In [0]:
X_train = trainData_Copy 
X_test = testData_Copy 

In [8]:
print(X_train.shape)
print(X_test.shape) 
print(X_test.head()) 

(65000, 595)
(173836, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]


In [0]:
#Select just Target Column
y_train = train_data.iloc[:, -1] 

In [10]:
print(y_train.shape) 
y_train.head() 

(65000,)


0    0
1    0
2    0
3    0
4    0
Name: QuoteConversion_Flag, dtype: int64

In [0]:
#CONSTRUCT DEFAULT DECISION TREE
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_train) 

In [12]:
#Hyperparameter tuning done for decision tree classifier 
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm) 

{'min_samples_split': 50, 'max_depth': 9}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_train) 

In [14]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above. 
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("Confusion Matrix after hypertuning for Decision Tree") 
print(confusion_matrix(y_train,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_train,clf_predict)) 

accuracy Score (training) after hypertuning for Decision Tree:0.919477
Confusion Matrix after hypertuning for Decision Tree
[[52175   563]
 [ 4671  7591]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     52738
           1       0.93      0.62      0.74     12262

    accuracy                           0.92     65000
   macro avg       0.92      0.80      0.85     65000
weighted avg       0.92      0.92      0.91     65000



In [15]:
#run cross-validation on best hyperparameters, get auc score 
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc") 
print("=== All AUC Scores ===")
print(clf_cv_score) 
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean()) 

=== All AUC Scores ===
[0.94422027 0.94327802 0.93758201 0.94422908 0.9398352  0.94837644
 0.9478046  0.93404864 0.94750633 0.92852124]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.941540183837722


In [16]:
#Testset prediction======================================================================
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"]) 
pred.head() 

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [17]:
# Getting Prediction using individual model  
pd.concat([X_test['QuoteNumber'], pred],axis=1).to_csv("/gdrive/My Drive/Colab Notebooks/result1.csv", index = None)  

res=pd.read_csv('/gdrive/My Drive/Colab Notebooks/result1.csv')
res.head() 

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Random Forest =============================================================
rfc = RandomForestClassifier() 
rfc.fit(X_train, y_train) 
rfc_predict=rfc.predict(X_train)

In [19]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc) 

{'n_estimators': 90, 'min_samples_split': 40, 'max_depth': 19}


In [20]:
#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc) 
rfc.fit(X_train,y_train) 
rfc_predict = rfc.predict(X_train) 
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_train,y_train))) 
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_train,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_train,rfc_predict)) 

accuracy Score (training) after hypertuning for Random Forest:0.938477
Confusion Matrix after hypertuning for Random Forest:
[[52520   218]
 [ 3781  8481]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     52738
           1       0.97      0.69      0.81     12262

    accuracy                           0.94     65000
   macro avg       0.95      0.84      0.89     65000
weighted avg       0.94      0.94      0.93     65000



In [21]:
#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean()) 

=== All AUC Scores ===
[0.94809033 0.94907843 0.94056379 0.95178988 0.94809574 0.95326376
 0.95280845 0.94331468 0.95396718 0.94231659]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9483288813722928


In [22]:
#Testset prediction======================================================================
pred2=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
pred2.head() 

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [23]:
# Getting Prediction using individual model 
pd.concat([X_test['QuoteNumber'], pred2],axis=1).to_csv("/gdrive/My Drive/Colab Notebooks/result2.csv", index = None)  

res2=pd.read_csv('/gdrive/My Drive/Colab Notebooks/result2.csv')
res2.head() 

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc = GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_train)

In [25]:
#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc) 

{'n_estimators': 50, 'learning_rate': 0.1}


In [26]:
#Construct Gradient Boosting using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_train)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_train,y_train)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_train,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_train,abc_predict)) 

accuracy Score (training) after hypertuning for Boosting:0.917031
Confusion Matrix after hypertuning for Boosting:
[[52402   336]
 [ 5057  7205]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     52738
           1       0.96      0.59      0.73     12262

    accuracy                           0.92     65000
   macro avg       0.93      0.79      0.84     65000
weighted avg       0.92      0.92      0.91     65000



In [27]:
#run cross-validation on best parameters, get auc score 
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean()) 

=== All AUC Scores ===
[0.95588055 0.95316014 0.94691524 0.95427653 0.95278208 0.95659947
 0.95813328 0.94836036 0.95806187 0.94850611]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.9532675626779721


In [28]:
#Testset prediction======================================================================
pred3=pd.DataFrame(abc.predict(X_test),columns=["QuoteConversion_Flag"])
pred3.head() 

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [29]:
# Getting Prediction using individual model 
pd.concat([X_test['QuoteNumber'], pred3],axis=1).to_csv("/gdrive/My Drive/Colab Notebooks/result3.csv", index = None)  

res3=pd.read_csv('/gdrive/My Drive/Colab Notebooks/result3.csv')
res3.head() 

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [30]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n") 
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5) 
X_res, y_res = sm.fit_resample(X_train, y_train) 
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [31]:
#STACKING MODELS ===================================================================== 
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ abc, rfc, clf] 
      
S_Train, S_Test = stacking(models,                     
                           X_res, y_res, X_test, 
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2) 

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93087930]
    fold  1:  [0.92718815]
    fold  2:  [0.93092987]
    fold  3:  [0.93148261]
    ----
    MEAN:     [0.93011998] + [0.00170916]
    FULL:     [0.93011996]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92349699]
    fold  1:  [0.92091824]
    fold  2:  [0.92395207]
    fold  3:  [0.92394822]
    ----
    MEAN:     [0.92307888] + [0.00126109]
    FULL:     [0.92307887]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.92228346]
    fold  1:  [0.91970471]
    fold  2:  [0.92233402]
    fold  3:  [0.91954895]
    ----
    MEAN:     [0.92096779] + [0.00134221]
    FULL:     [0.92096780]



In [32]:
# STACKING - Random Forest Model =========================================================
model2 = RandomForestClassifier() 

model2 = model2.fit(S_Train, y_res) 
y_pred2 = model2.predict(S_Train) 
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_res, y_pred2)) 

print("Confusion Matrix after stacking for Random Forest:") 
print(confusion_matrix(y_res,y_pred2)) 

print("=== Classification Report ===") 
print(classification_report(y_res,y_pred2)) 

Final prediction score for ensemble methods: [0.93119446]
Confusion Matrix after stacking for Random Forest:
[[51704  1034]
 [ 4409 21960]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     52738
           1       0.96      0.83      0.89     26369

    accuracy                           0.93     79107
   macro avg       0.94      0.91      0.92     79107
weighted avg       0.93      0.93      0.93     79107



In [33]:
#Hyperparameter tuning for random forest 
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
model2_random = RandomizedSearchCV(model2,parameters,n_iter=15) 
model2_random.fit(S_Train, y_res) 
grid_parm_model2=model2_random.best_params_ 
print(grid_parm_model2) 

{'n_estimators': 130, 'min_samples_split': 20, 'max_depth': 1}


In [34]:
# STACKING - Random Forest Model =========================================================
model2 = RandomForestClassifier(**grid_parm_model2) 

model2 = model2.fit(S_Train, y_res) 
y_pred2 = model2.predict(S_Train)  
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_res, y_pred2))  

print("Confusion Matrix after stacking for Random Forest:") 
print(confusion_matrix(y_res,y_pred2))  

print("=== Classification Report ===") 
print(classification_report(y_res,y_pred2)) 

Final prediction score for ensemble methods: [0.93119446]
Confusion Matrix after stacking for Random Forest:
[[51704  1034]
 [ 4409 21960]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     52738
           1       0.96      0.83      0.89     26369

    accuracy                           0.93     79107
   macro avg       0.94      0.91      0.92     79107
weighted avg       0.93      0.93      0.93     79107



In [35]:
#Testset prediction====================================================================== 
predFinalFR=pd.DataFrame(model2.predict(S_Test),columns=["QuoteConversion_Flag"]) 
predFinalFR.head() 

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [36]:
# Getting Prediction using stacking model 
pd.concat([X_test['QuoteNumber'], predFinalFR],axis=1).to_csv("/gdrive/My Drive/Colab Notebooks/resultFinal.csv", index = None)  

resf=pd.read_csv('/gdrive/My Drive/Colab Notebooks/resultFinal.csv')
resf.head() 

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [37]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n") 
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=1) 
X_res, y_res = sm.fit_resample(X_train, y_train) 
print('Resampled dataset shape %s' % Counter(y_res)) 

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 52738})


In [38]:
#STACKING MODELS ===================================================================== 
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [abc, rfc, clf] 
      
S_Train, S_Test = stacking(models,                     
                           X_res, y_res, X_test, 
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2) 

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.94334256]
    fold  1:  [0.94087755]
    fold  2:  [0.94209109]
    fold  3:  [0.94247033]
    ----
    MEAN:     [0.94219538] + [0.00088589]
    FULL:     [0.94219538]

model  1:     [RandomForestClassifier]
    fold  0:  [0.94220486]
    fold  1:  [0.94190148]
    fold  2:  [0.94334256]
    fold  3:  [0.94303917]
    ----
    MEAN:     [0.94262202] + [0.00058873]
    FULL:     [0.94262202]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.93420304]
    fold  1:  [0.93181387]
    fold  2:  [0.93435473]
    fold  3:  [0.93371004]
    ----
    MEAN:     [0.93352042] + [0.00101370]
    FULL:     [0.93352042]



In [39]:
# STACKING - Random Forest Model =========================================================
model2 = RandomForestClassifier() 

model2 = model2.fit(S_Train, y_res) 
y_pred2 = model2.predict(S_Train) 
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_res, y_pred2)) 

print("Confusion Matrix after stacking for Random Forest:") 
print(confusion_matrix(y_res,y_pred2)) 

print("=== Classification Report ===") 
print(classification_report(y_res,y_pred2)) 

Final prediction score for ensemble methods: [0.94770374]
Confusion Matrix after stacking for Random Forest:
[[50824  1914]
 [ 3602 49136]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.96      0.95     52738
           1       0.96      0.93      0.95     52738

    accuracy                           0.95    105476
   macro avg       0.95      0.95      0.95    105476
weighted avg       0.95      0.95      0.95    105476



In [40]:
#Hyperparameter tuning for random forest 
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
model2_random = RandomizedSearchCV(model2,parameters,n_iter=15) 
model2_random.fit(S_Train, y_res) 
grid_parm_model2=model2_random.best_params_ 
print(grid_parm_model2) 

{'n_estimators': 50, 'min_samples_split': 60, 'max_depth': 1}


In [41]:
# STACKING - Random Forest Model =========================================================
model2 = RandomForestClassifier(**grid_parm_model2) 

model2 = model2.fit(S_Train, y_res) 
y_pred2 = model2.predict(S_Train)  
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_res, y_pred2))  

print("Confusion Matrix after stacking for Random Forest:") 
print(confusion_matrix(y_res,y_pred2))  

print("=== Classification Report ===") 
print(classification_report(y_res,y_pred2)) 

Final prediction score for ensemble methods: [0.94758049]
Confusion Matrix after stacking for Random Forest:
[[51251  1487]
 [ 4042 48696]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     52738
           1       0.97      0.92      0.95     52738

    accuracy                           0.95    105476
   macro avg       0.95      0.95      0.95    105476
weighted avg       0.95      0.95      0.95    105476



In [42]:
#Testset prediction====================================================================== 
predFinalFR=pd.DataFrame(model2.predict(S_Test),columns=["QuoteConversion_Flag"]) 
predFinalFR.head() 

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [43]:
# Getting Prediction using stacking model 
pd.concat([X_test['QuoteNumber'], predFinalFR],axis=1).to_csv("/gdrive/My Drive/Colab Notebooks/resultFinal2.csv", index = None)  

resfff=pd.read_csv('/gdrive/My Drive/Colab Notebooks/resultFinal2.csv')
resfff.head() 

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
